In [2]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4

In [4]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from dotenv import load_dotenv
from langchain_community.llms import LlamaCpp
import ollama
from langchain_community.embeddings import OllamaEmbeddings

In [6]:
import os 
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [20]:
ollama_emb = OllamaEmbeddings(
    model="mxbai-embed-large",
)
llm = Ollama(model = 'llama3') 
chat_model = ChatOllama(model = 'llama3')


In [8]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=ollama_emb)
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
 
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


In [11]:
qa_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [12]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

According to the text, common ways of doing Task Decomposition include:

1. Using LLM with simple prompting like "Steps for XYZ.\n1." or "What are the subgoals for achieving XYZ?"
2. Providing task-specific instructions, such as "Write a story outline." for writing a novel
3. Utilizing human inputs

These techniques help break down complex tasks into smaller, manageable steps, allowing for more effective planning and execution.


In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [14]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Task decomposition is a technique that breaks down complex tasks into smaller, simpler steps. This allows for more manageable problem-solving and sheds light on the model's thinking process. It can be achieved through techniques like Chain of Thought (CoT) or Tree of Thoughts."

In [15]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Task decomposition is done by LLM with simple prompting, using task-specific instructions, or with human inputs. Common methods include Chain of Thought (CoT), Tree of Thoughts, and few-shot examples to guide LLM in task parsing and planning. These techniques help decompose hard tasks into smaller and simpler steps, making problem-solving more manageable.'

In [16]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is a technique that breaks down complex tasks into smaller, simpler steps. This allows for more manageable problem-solving and sheds light on the model's thinking process. It can be achieved through techniques like Chain of Thought (CoT) or Tree of Thoughts.

User: What are common ways of doing it?

AI: Task decomposition is done by LLM with simple prompting, using task-specific instructions, or with human inputs. Common methods include Chain of Thought (CoT), Tree of Thoughts, and few-shot examples to guide LLM in task parsing and planning. These techniques help decompose hard tasks into smaller and simpler steps, making problem-solving more manageable.



In [25]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter



### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=ollama_emb)
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    chat_model, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(chat_model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [26]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task Decomposition refers to a technique used in complex tasks, where a hard task is broken down into smaller and more manageable sub-tasks. This approach helps models like LLM-powered autonomous agents plan ahead and think step-by-step, utilizing more test-time computation to achieve better performance on difficult tasks.'

In [27]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'According to the provided context, common ways of Task Decomposition include:\n\n1. Using Large Language Models (LLMs) with simple prompting, such as "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?"\n2. Utilizing task-specific instructions, like "Write a story outline." for writing a novel\n3. Incorporating human inputs'

# Agents

In [28]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]

In [32]:
from langchain_openai import ChatOpenAI
chat_model = ChatOpenAI(
    api_key="ollama",
    model="llama3",
    base_url="http://localhost:11434/v1",
)

In [33]:
tool.invoke("task decomposition")

'Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\n\nFig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by st

In [34]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(chat_model, tools)

In [35]:
query = "What is Task Decomposition?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Task decomposition is a technique used in project management, problem-solving, and goal-setting. It involves breaking down a complex task or project into smaller, more manageable sub-tasks or components, often referred to as "sub-tasks" or "child tasks".\n\nThe idea behind task decomposition is that a large, overwhelming task can be made more feasible by dividing it into smaller, less daunting pieces. This approach helps to:\n\n1. Simplify the task: By breaking down the task into smaller parts, it becomes easier to understand and tackle.\n2. Focus on specific goals: Sub-tasks can have clear objectives and deadlines, allowing teams or individuals to concentrate on a specific outcome.\n3. Improve progress tracking: Task decomposition makes it easier to monitor progress, as each sub-task can be measured independently.\n4. Enhance collaboration: By having multiple sub-tasks, team members can work simultaneously on different aspects of the project,

In [37]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = create_react_agent(chat_model, tools, checkpointer=memory)

In [38]:
config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm bob")]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content="Hi Bob! It's nice to meet you. How's your day going so far?", response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 10, 'total_tokens': 29}, 'model_name': 'llama3', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-9475a932-1a58-4bfd-b686-3a9c3cecc2d6-0', usage_metadata={'input_tokens': 10, 'output_tokens': 19, 'total_tokens': 29})]}}
----


In [39]:
query = "What is Task Decomposition?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Task decomposition is a planning and problem-solving technique that breaks down complex tasks or projects into smaller, manageable, and more tangible pieces (tasks). This process helps to clarify the overall task, identify key steps, and create a roadmap for completion.\n\nIn task decomposition, you:\n\n1. Define the high-level goal or objective.\n2. Identify the most important tasks required to achieve that goal.\n3. Break down each of those tasks into smaller, more specific subtasks (usually 3-5 per task).\n4. Continue this process until you have a detailed plan with clear actions and deliverables.\n\nTask decomposition has many benefits, including:\n\n1. Improved understanding: By breaking down complex tasks, you gain clarity on what needs to be done.\n2. Reduced overwhelm: You\'ll feel more in control as you focus on one task at a time.\n3. Increased efficiency: You can start working on individual tasks sooner, without waiting for the enti

In [40]:
query = "What according to the blog post are common ways of doing it? redo the search"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content="I dug up the info on common approaches to task decomposition. According to various sources, here are some popular methods:\n\n1. **Divide and Conquer**: Break down a large project or task into smaller, independent tasks that can be worked on simultaneously. This approach is particularly useful when you have multiple team members involved.\n2. **Work Breakdown Structure (WBS)**: A hierarchical decomposition technique that starts with the overall goal and progressively breaks down larger tasks into smaller, more manageable ones. WBS is commonly used in construction, manufacturing, and project management.\n3. **Task Segmentation**: Divide a task into logical chunks based on specific criteria, such as:\n\t* Time-based segmentation (e.g., breaking a project into daily or weekly tasks).\n\t* Function-based segmentation (e.g., dividing tasks by responsibility or skill set).\n\t* Priority-based segmentation (e.g., categorizing tasks by urgency or impo

In [41]:
import bs4
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

memory = SqliteSaver.from_conn_string(":memory:")
chat_model = ChatOpenAI(
    api_key="ollama",
    model="llama3",
    base_url="http://localhost:11434/v1",
)


### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=ollama_emb)
retriever = vectorstore.as_retriever()


### Build retriever tool ###
tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]


agent_executor = create_react_agent(chat_model, tools, checkpointer=memory)